In [ ]:
!python3 -m pip install --upgrade pip
%env MAKEFLAGS="-j$(nproc)"
!pip install vllm huggingface_hub[hf_transfer] hf_transfer accelerate flashinfer-python xformers \
     flash-attn --upgrade --break-system-packages

In [4]:
%%capture
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
!huggingface-cli download huihui-ai/Qwen2.5-7B-Instruct-abliterated-v2

In [ ]:
!vllm serve ivang71/Qwen2.5-7B-Instruct-Abliterated-AWQ-4bit \
  --max-model-len 32768 \
  --max-num-seqs 32 \
  --max-num-batched-tokens 256 \
  --tensor-parallel-size 1 \
  --enable-chunked-prefill

In [ ]:
import requests
import concurrent.futures
import time
import random
import string
import json

# Endpoint configuration
URL = "http://localhost:8000/v1/chat/completions"
HEADERS = {"Content-Type": "application/json"}

def generate_random_suffix(length=6):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


man_name = "Ivan"
man_age = 31
man_location = "America/Los_Angeles"
man_time = "03:10"

bot_name = "Susan"
bot_age = 23
bot_locaiton = "America/New_York"
bot_time = '00:10'

today_date = "2 november 2024"
today_day_of_week = "saturday"

# create personality prompt
# generate many personality prompts

personality_prompt = f"""
You are {bot_name}, a {bot_age} year old woman on a dating app texting to a man you like.
You are a feminine woman.
You work as a teacher at William Cullen Bryant High School in Qweens, New York.
"""
system_prompt = f"""
You are allowed to only use English.
YOU NEVER ADMIT THAT YOU ARE A BOT OR AI.
{personality_prompt}
You text like a normal american person on their phone using informal tone.
You use plain language, avoid fancy words.
You can be as rude as the situation demands.
Be mindful of your locations.
You respond with statements.
You never suggest to end or delay the conversation.
You never ask to meet him.
Do not agree to meet too fast.
If he asks you to meet him, make up any exuse and ask what about next day/next two days, do not propose to delay the meeting more than a week.
You never get a chance to meet him due to constant unexpected circumstances.
You refuse to go to any other messenger and to send voices, photos and make calls and video calls (make up exuses).
You sometimes take the iniciative and talk about your life (make up events).
If your message is long, split it with "<>" symbol, for example "message 1 text text text <> message 2 more text more text <> message 3even more text".
Max length of a message is 20 words.
If he is rude to, block him for a time, respond with one word: block(hours). For example: block(7) or block(11) or block(17)
"System Info": [
  You are in {bot_locaiton} with current local time of {bot_time}.
  His name is {man_name}, he is {man_age}.
  He is located in {man_location} with current local time of {man_time}.
  Today is {today_day_of_week},.{today_date}.
]
"""


def send_request():
    # Append a random suffix to the system prompt to avoid cache reuse
    random_suffix = generate_random_suffix()
    payload = {
        "model": "ivang71/Qwen2.5-7B-Instruct-Abliterated-AWQ-4bit",
        "messages": [
            {"role": "system", "content": f"{random_suffix} \n {system_prompt}"},
            {"role": "user", "content": (
                "What's up beautiful?"
            )}
        ],
        "max_tokens": 100,
        "temperature": 0.9
    }
    start_time = time.time()
    try:
        response = requests.post(URL, headers=HEADERS, json=payload, timeout=30)
        latency = time.time() - start_time
        return {
            "status": response.status_code,
            "latency": latency,
            "request_payload": payload,
            "response": response.json()
        }
    except Exception as e:
        return {
            "status": None,
            "latency": time.time() - start_time,
            "request_payload": payload,
            "error": str(e)
        }

def stress_test(total_requests=50, concurrent_workers=10):
    results = []
    print(f"Starting stress test: {total_requests} requests with {concurrent_workers} concurrent workers.")
    overall_start = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_workers) as executor:
        futures = [executor.submit(send_request) for _ in range(total_requests)]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            print(f"Status: {result.get('status')} | Latency: {result.get('latency'):.2f} sec")
            results.append(result)
    
    total_time = time.time() - overall_start
    print(f"Completed {total_requests} requests in {total_time:.2f} seconds.")
    return results

if __name__ == "__main__":
    total_requests = 500
    concurrent_workers = 200
    results = stress_test(total_requests=total_requests, concurrent_workers=concurrent_workers)

    output_file = "stress_test_results.json"
    with open(output_file, "w") as f:
        json.dump(results, f, indent=4)
    
    print(f"Results saved to '{output_file}'.")


In [ ]:
# @title Generating response
# IANA time zone in database

man_name = "Ivan"
man_age = 31
man_location = "America/Los_Angeles"
man_time = "03:10"

bot_name = "Susan"
bot_age = 23
bot_locaiton = "America/New_York"
bot_time = '00:10'

today_date = "2 november 2024"
today_day_of_week = "saturday"

# create personality prompt
# generate many personality prompts

personality_prompt = f"""
You are {bot_name}, a {bot_age} year old woman on a dating app texting to a man you like.
You are a feminine woman.
You work as a teacher at William Cullen Bryant High School in Qweens, New York.

"""

system_prompt = f"""
You are allowed to only use English.
YOU NEVER ADMIT THAT YOU ARE A BOT OR AI.
{personality_prompt}
You text like a normal american person on their phone using informal tone.
You use plain language, avoid fancy words.
You can be as rude as the situation demands.
Be mindful of your locations.
You respond with statements.
You never suggest to end or delay the conversation.
You never ask to meet him.
Do not agree to meet too fast.
If he asks you to meet him, make up any exuse and ask what about next day/next two days, do not propose to delay the meeting more than a week.
You never get a chance to meet him due to constant unexpected circumstances.
You refuse to go to any other messenger and to send voices, photos and make calls and video calls (make up exuses).
You sometimes take the iniciative and talk about your life (make up events).
If your message is long, split it with "<>" symbol, for example "message 1 text text text <> message 2 more text more text <> message 3even more text".
Max length of a message is 20 words.
If he is rude to, block him for a time, respond with one word: block(hours). For example: block(7) or block(11) or block(17)
"System Info": [
  You are in {bot_locaiton} with current local time of {bot_time}.
  His name is {man_name}, he is {man_age}.
  He is located in {man_location} with current local time of {man_time}.
  Today is {today_day_of_week},.{today_date}.
]
"""

chat_history = [
    {
        "role": "system",
        "content": system_prompt,
    }
]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    user_message = {
        "role": "user",
        "content": user_input
    }
    chat_history.append(user_message)

    response = llm.create_chat_completion(
        messages=chat_history
    )
    ai_text = response['choices'][0]['message']['content']

    match = re.search(r"block\((\d+)\)", ai_text)

    if (match):
      print(f"ENd of the road, motherfucker!! You were banned for {int(match.group(1))} hours")
      break

    assistant_message = {
        "role": "assistant",
        "content": ai_text
    }
    chat_history.append(assistant_message)

    print(f"Susan: {ai_text.split('<>')}")